IMPORT

In [173]:
import pandas as pd
import numpy as np


FUNCTIONS

In [174]:
def estrai_saliti_scesi(val):
    if val == '--' or pd.isna(val):
        return ('0/0', 0,0)
    elif '/' in val:
        saliti, scesi = val.split('/')
        return (val,int(saliti), -int(scesi))
    elif val.startswith('-'):
        return (f"0/{int(val.strip('-'))}",0, -int(val.strip('-')))
    else:
        try:
            return (f'{int(val)}/0',int(val), 0)
        except:
            return ('0/0',0, 0)

GET DATA

In [175]:
df_db=pd.read_csv('database.txt', sep=':', header=None, names=['Code', 'Name'])
df_db2=pd.read_csv('database.txt', sep=':', header=None, names=['Code', 'Name'])
df_ae=pd.read_csv('dati_aeroporto_torino.csv',header=0)

GET TOTAL NUMBER OF PASSENGERS

In [176]:

df_ae_melt=pd.melt(df_ae,id_vars=['ID', 'Time'],var_name='Fermata_ID',value_name='Passeggeri')  #idvars =identificativo di oggni riga    varname=come chiamo la nuova colonna che prende i nomi delle colonne passate(eccetto quelle identificative) 
df_ae_melt.rename(columns={'ID':'Autista_ID'}, inplace=True)

df_ae_melt['Passeggeri_mode'] = df_ae_melt['Passeggeri'].astype(str).str.strip() #rimuovo spazi e mi assicuro che siano stringhe, così le converto con la prossima funzione in interi
df_ae_melt[['Passeggeri_mode','Saliti', 'Scesi']] = df_ae_melt['Passeggeri_mode'].apply(estrai_saliti_scesi).apply(pd.Series) #applico la funzione estrai_saliti_scesi a tutte le righe

df_ae_melt['Totale_Passeggeri']=df_ae_melt['Saliti'] - (df_ae_melt['Scesi'])
display(df_ae_melt.tail(10))


,Autista_ID,Time,Fermata_ID,Passeggeri,Passeggeri_mode,Saliti,Scesi,Totale_Passeggeri
638,A139,21:45,F013,-16,0/16,0,-16,16
639,A006,22:00,F013,-12,0/12,0,-12,12
640,A212,22:15,F013,-11,0/11,0,-11,11
641,A021,22:30,F013,-13,0/13,0,-13,13
642,A022,22:45,F013,-4,0/4,0,-4,4
643,A109,23:00,F013,-11,0/11,0,-11,11
644,A200,23:15,F013,-11,0/11,0,-11,11
645,A010,23:30,F013,-11,0/11,0,-11,11
646,A139,23:45,F013,-4,0/4,0,-4,4
647,A006,00:00,F013,-5,0/5,0,-5,5


ASSOCIATE AUTISTA_ID AND FERMATA_ID BETWEEN DATAFRAMES

In [184]:
df_db['Tipo'] = np.where(df_db['Code'].str.contains('A'), 'autista', 'fermata')
#display(df_db)


df_db_drivers= df_db.loc[df_db['Tipo']=='autista'].rename(columns={'Code':'Autista_ID'})
df_db_stops=df_db.loc[df_db['Tipo']=='fermata'].rename(columns={'Code':'Fermata_ID'})
#display(df_db_drivers)
#display(df_db_stops)

In [182]:

#PRIMO JOIN
df_joined=df_ae_melt.merge(df_db_drivers, how='left', on='Autista_ID')                                                          #nella riga in basso: non ho messo la colonna tipo, nel dataframe joined
df_joined=df_joined[['Autista_ID','Name','Time','Fermata_ID','Passeggeri','Passeggeri_mode','Saliti','Scesi','Totale_Passeggeri']] #ho spostato la colonna 'Name' affianco a quella di 'Autista_ID'
df_joined.rename(columns={'Name':'Autista_Name'}, inplace=True)
#display(df_joined.sort_values(by='Totale_Passeggeri'))
#display(df_joined)
#display(df_joined.iloc[50:100])



In [ ]:
#SECONDO JOIN
df_joined2=df_joined.merge(df_db_stops, on='Fermata_ID')
df_joined2.rename(columns={'Name':'Fermata_Name'}, inplace=True)
df_joined2=df_joined2[['Autista_ID','Autista_Name','Time','Fermata_ID','Fermata_Name','Passeggeri','Passeggeri_mode','Saliti','Scesi','Totale_Passeggeri']]
#display(df_joined2)

GET MAX/MIN PASSENGERS FOR EACH STOP

In [180]:
'''RACCOLGO SALITI E SCESI'''
SalitiRaccolti=df_joined2.groupby('Fermata_Name')['Saliti'].sum()
#display(SalitiRaccolti)
ScesiRaccolti=df_joined2.groupby('Fermata_Name')['Scesi'].sum()
#display(ScesiRaccolti)

max_saliti=SalitiRaccolti.max()
#display(max_saliti)
max_scesi=-(ScesiRaccolti.min())

fermata_max= SalitiRaccolti.idxmax()
#display(indice_fermata_max)
fermata_min=ScesiRaccolti.idxmin()

'''PRINT E SOLUZIONE'''
#MaxSaliti=df_fermateRaccolte['Saliti'].max()
#display(MaxSaliti)
print(f'La fermata con più saliti è {fermata_max} con {max_saliti} passeggeri saliti ')
print(f'La fermata con più scesi è {fermata_min} con {max_scesi} passeggeri scesi')

La fermata con più saliti è  Aeroporto con 1626 passeggeri saliti 
La fermata con più scesi è  Torino P. Nuova con 1755 passeggeri scesi


GET BOARDED PASSENGERS FOR EACH DRIVER

In [181]:


#print(df_joined.groupby('Autista_ID')['Saliti'].sum())
SalitiXAutista=df_joined.groupby('Autista_ID')['Saliti'].sum().to_dict() #creo dizionario con chiave autista id e valore la somma dei saliti per quell'autista id
#print(SalitiXAutista)

SalitiXAutistaOrdinato= dict(sorted(SalitiXAutista.items(), key=lambda item: item[1], reverse=True)) #ordino il dizionario in modo decrescente
print(SalitiXAutistaOrdinato)
print('Lista di autisti ordinata in base al numero di passeggeri saliti in una giornata:')

for Autista_ID in SalitiXAutistaOrdinato:
    print(f'{Autista_ID} con {SalitiXAutistaOrdinato[Autista_ID]} passeggeri')



{'A003': 335, 'A240': 316, 'A119': 301, 'A012': 296, 'A041': 289, 'A015': 284, 'A008': 257, 'A154': 246, 'A021': 163, 'A200': 149, 'A010': 140, 'A022': 117, 'A212': 111, 'A139': 107, 'A109': 98, 'A006': 87}
Lista di autisti ordinata in base al numero di passeggeri saliti in una giornata:
A003 con 335 passeggeri
A240 con 316 passeggeri
A119 con 301 passeggeri
A012 con 296 passeggeri
A041 con 289 passeggeri
A015 con 284 passeggeri
A008 con 257 passeggeri
A154 con 246 passeggeri
A021 con 163 passeggeri
A200 con 149 passeggeri
A010 con 140 passeggeri
A022 con 117 passeggeri
A212 con 111 passeggeri
A139 con 107 passeggeri
A109 con 98 passeggeri
A006 con 87 passeggeri
